In [4]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option("display.max_columns", None)
pd.option_context('display.max_rows', None, 'display.max_columns', None)
import dash
import dash
from dash import dcc
from dash import html
import plotly.express as px
from plotly import figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import csv
import os
import pathlib as path
import plotly #https://plotly.com/python/getting-started/ /////// pip install plotly
import numpy as np
import scipy
from scipy import stats


In [327]:

toplaner_agg = path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\TopLaneStats_agg_50games_majorLeague.csv')
toplaner_agg_winloss =  path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\TopLaneStats_agg_Win-Loss_majorLeague.csv')
df_ta = pd.read_csv(toplaner_agg)
df_wl = pd.read_csv(toplaner_agg_winloss)

In [328]:
dft = df_wl[['player', 'games','result', 'avg(earnedgoldshare)']]
tat = df_ta[['player','winPercentage']]
dft2 = dft.merge(tat, on='player')

def func(result, winPercentage):
    if result == 0:
        return 100-winPercentage
    else:
        return winPercentage



dft2['winPercentage'] = dft2.apply(lambda x: func(x['result'], x['winPercentage']), axis=1)
dft2.reset_index(drop=True, inplace=True)




In [377]:
mymean = np.mean(dft2['avg(earnedgoldshare)'])
mystd = np.std(dft2['avg(earnedgoldshare)'])
mypd = scipy.stats.norm.pdf((dft2['avg(earnedgoldshare)']), mymean, mystd)
mult = [-3,-2,-1,0,1,2,3]
distribution = np.linspace(mymean - 3*mystd, mymean + 3*mystd, 185)
stdvals = [((i*mystd)+mymean).round(3) for i in mult]
print(stdvals)
#px.scatter(df_ta['avg(earnedgoldshare)'], x=x_axis, y=df_ta['avg(earnedgoldshare)'], hover_data=[df_ta['player']])
#px.hist(df_ta['avg(earnedgoldshare)'], x, stats.norm.pdf(x, mymean, mystd), hover_data=[df_ta['player']])
#fig = px.histogram(dft2['avg(earnedgoldshare)'], x=distribution, nbins = 15, labels={'x':'EarnedGoldShare', 'y':'count'}, pattern_shape=dft2['result'])

winners = dft2.loc[dft2['result']==1]['avg(earnedgoldshare)']

losers = dft2.loc[dft2['result']==0]['avg(earnedgoldshare)']

text = list(dft2.player)

hist_data = [winners.round(4), losers.round(4)]
group_labels = ['Won', 'Lost']
colors = ['blue', 'orange']

counts, bins = np.histogram(winners)
fig = ff.create_distplot(hist_data, 
                        group_labels, 
                        bin_size = 0.009, 
                        colors = colors,
                        curve_type='normal',
                        
                        )


fig.update_layout(bargap=0.02,
                title_text = 'Avg Earned Gold Share By Result')
fig.update_xaxes(dtick=stdvals)

fig.show()


[0.188, 0.199, 0.21, 0.221, 0.232, 0.243, 0.254]


In [354]:
dft2

winners = dft2.loc[dft2['result']==1]['avg(earnedgoldshare)']
losers = dft2.loc[dft2['result']==0]['avg(earnedgoldshare)']
losers.round(3)




0      0.227
2      0.227
4      0.214
6      0.217
8      0.217
       ...  
360    0.215
362    0.241
364    0.221
366    0.209
368    0.223
Name: avg(earnedgoldshare), Length: 185, dtype: float64

In [382]:
dft2.loc[dft2['player']=='Solo']

,player,games,result,avg(earnedgoldshare),winPercentage
290,Solo,64,0,0.231103,53.3333
291,Solo,56,1,0.219515,46.6667


In [25]:
toplaneFile = path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\TopLaneStatsRaw.csv')
topstats = pd.read_csv(toplaneFile)
topstats2 = topstats[topstats['patch'] > 8.01]

In [30]:
solostats = topstats2.loc[topstats2['player']=='Solo']
toplanerstats = topstats2.loc[topstats2['player']!='Solo']
#120 games
champs = solostats['champion'].value_counts()
topchamps = toplanerstats['champion'].value_counts()
champs

#toplanerstats.info()
#topchamps


Aatrox         21
Ornn           18
Sett           13
Renekton       11
Gangplank      10
Shen            7
Jayce           5
Gnar            5
Mordekaiser     4
Akali           4
Nocturne        3
Sion            3
Vladimir        3
Maokai          2
Urgot           2
Malphite        2
Volibear        2
Cho'Gath        1
Yorick          1
Kennen          1
Rumble          1
Lee Sin         1
Name: champion, dtype: int64

In [59]:
cprf = path.Path(r'F:\LeagueStats\scraping\LeagueDataAnalysis\workingTables\champion play rate.csv')
cpr = pd.read_csv(cprf)

#Test procedure
l = cpr.loc[cpr['player']=='Licorice'].sort_values('games', ascending=False )
total_games = sum(l['games'])
l['playrate'] = l['games'].apply(lambda x: 100*(x/total_games))


cpr = cpr.sort_values('player')
players = cpr['player'].unique() #array of players
agg_cpr = pd.DataFrame(columns = ['player', 'champion', 'games', 'playrate'])
for player in players:
    p = cpr.loc[cpr['player']==player]
    total_games = sum(p['games'])
    p['playrate'] = p['games'].apply(lambda x: 100*(x/total_games))
    agg_cpr = pd.concat([agg_cpr, p])











In [65]:
agg_cpr



,player,champion,games,playrate
205,369,Wukong,6,2.371542
2925,369,Maokai,3,1.185771
4427,369,Gragas,4,1.581028
2680,369,Neeko,3,1.185771
3019,369,Malphite,4,1.581028
...,...,...,...,...
2554,zs,Ornn,3,4.000000
4216,zs,Irelia,1,1.333333
2584,zs,Nocturne,4,5.333333
2030,zs,Renekton,15,20.000000
